In [6]:
#####################
### Data Analysis ###
#####################

## Please set your working directory to the data/ folder

# Clear the workspace
rm(list = ls())

# Load necessary packages
library(dplyr) # data manipulation
library(Synth) # models

# Load data
df <- read.csv("df.csv", header = TRUE)

# Prepare dataset
df$state <- as.character(df$state) # required by dataprep()

# Plot: Homicide rates for Sao Paulo and Brazil (average)
df1 <- df %>%
        mutate(homicide.sp = ifelse(homicide.rates & state == "São Paulo", homicide.rates, NA)) %>%
        select(year, homicide.sp)

df2 <- df %>%
        mutate(homicide.rates1 = ifelse(homicide.rates & state != "São Paulo", homicide.rates, NA)) %>%
        group_by(year) %>%
        summarise(homicide.br = mean(homicide.rates1, na.rm = TRUE))

setEPS()
postscript(file    = "br.eps",
           horiz   = FALSE,
           onefile = FALSE,
           width   = 7,     # 17.8 cm
           height  = 5.25)  # 13.3 cm

plot(x = df1$year,
     y = df1$homicide.sp,
     type = "l",
     ylim = c(0, 60),
     xlim = c(1990, 2009),
     xlab = "Year",
     ylab = "Homicide Rates",
     cex = 3,
     lwd = 2,
     xaxs = "i",
     yaxs = "i"
)

lines(df2$year,
      df2$homicide.br,
      lty = 2,
      cex = 3,
      lwd = 2)

arrows(1997, 50, 1999, 50,
       col    = "black",
       length = .1)

text(1995, 50,
     "Policy Change",
     cex = .8)

abline(v   = 1999,
       lty = 2)

legend(x = "bottomleft",
       legend = c("São Paulo",
                  "Brazil (average)"),
       lty    = c("solid", "dashed"),
       cex    = .8,
       bg     = "white",
       lwdc(2, 2)
)

invisible(dev.off())

# Prepare data for synth
dataprep.out <-
        dataprep(df,
                 predictors = c("state.gdp.capita",
                                "state.gdp.growth.percent",
                                "population.projection.ln",
                                "years.schooling.imp"
                                ),
                 special.predictors = list(
                         list("homicide.rates", 1990:1998, "mean"),
                         list("proportion.extreme.poverty", 1990:1998, "mean"),
                         list("gini.imp", 1990:1998, "mean")
                         ),
                 predictors.op = "mean",
                 dependent     = "homicide.rates",
                 unit.variable = "code",
                 time.variable = "year",
                 unit.names.variable   = "state",
                 treatment.identifier  = 35,
                 controls.identifier   = c(11:17, 21:27, 31:33, 41:43, 50:53),
                 time.predictors.prior = c(1990:1998),
                 time.optimize.ssr     = c(1990:1998),
                 time.plot             = c(1990:2009)
                 )

# Run synth
synth.out <- synth(dataprep.out)

# Get result tables
print(synth.tables   <- synth.tab(
        dataprep.res = dataprep.out,
        synth.res    = synth.out)
      )

# Plot: Main model
setEPS()
postscript(file    = "trends.eps",
           horiz   = FALSE,
           onefile = FALSE,
           width   = 7,     # 17.8 cm
           height  = 5.25)  # 13.3 cm

path.plot(synth.res    = synth.out,
          dataprep.res = dataprep.out,
          Ylab         = c("Homicide Rates"),
          Xlab         = c("Year"),
          Legend       = c("São Paulo","Synthetic São Paulo"),
          Legend.position = c("bottomleft")
)

abline(v   = 1999,
       lty = 2)

arrows(1997, 50, 1999, 50,
       col    = "black",
       length = .1)

text(1995, 50,
     "Policy Change",
     cex = .8)

invisible(dev.off())

# Main model: gaps plot
setEPS()
postscript(file    = "gaps.eps",
           horiz   = FALSE,
           onefile = FALSE,
           width   = 7,
           height  = 5.25)

gaps.plot(synth.res    = synth.out,
          dataprep.res = dataprep.out,
          Ylab         = c("Gap in Homicide Rates"),
          Xlab         = c("Year"),
          Ylim         = c(-30, 30),
          Main         = ""
)

abline(v   = 1999,
       lty = 2)

arrows(1997, 20, 1999, 20,
       col    = "black",
       length = .1)

text(1995, 20,
     "Policy Change",
     cex = .8)

invisible(dev.off())

## Calculating how many lives were saved during the treatment period

# Weights below retrieved form dataprep.out
# State Code  State Weight  State Name        State Abbreviation
# 42          0.274         Santa Catarina    SC
# 53          0.210         Distrito Federal  DF
# 32          0.209         Espirito Santo    ES
# 33          0.169         Rio de Janeiro    RJ
# 14          0.137         Roraima           RR
# 14          0.001         Pernambuco        PB
# 35          treat         Sao Paulo         SP

# Get years after policy change
df.2 <- df[which(df$year >= 1999),]

# Calculate total number of deaths in SP
num.deaths.sp <- sum( (df.2$homicide.rates[which(df.2$abbreviation == "SP")])/100000 * (df.2$population.projection[which(df.2$abbreviation == "SP")]))

#Calculate estimated number of deaths in Synthetic São Paulo
num.deaths.synthetic.sp <- sum( (0.274 * (df.2$homicide.rates[which(df.2$abbreviation == "SC")])/100000 * (df.2$population.projection[which(df.2$abbreviation == "SP")]))
                                + (0.210 * (df.2$homicide.rates[which(df.2$abbreviation == "DF")])/100000 * (df.2$population.projection[which(df.2$abbreviation == "SP")]))
                                + (0.209 * (df.2$homicide.rates[which(df.2$abbreviation == "ES")])/100000 * (df.2$population.projection[which(df.2$abbreviation == "SP")]))
                                + (0.169 * (df.2$homicide.rates[which(df.2$abbreviation == "RJ")])/100000 * (df.2$population.projection[which(df.2$abbreviation == "SP")]))
                                + (0.137 * (df.2$homicide.rates[which(df.2$abbreviation == "RR")])/100000 * (df.2$population.projection[which(df.2$abbreviation == "SP")]))
                                + (0.001 * (df.2$homicide.rates[which(df.2$abbreviation == "PB")])/100000 * (df.2$population.projection[which(df.2$abbreviation == "SP")]))
                                )

lives.saved <- num.deaths.synthetic.sp - num.deaths.sp
lives.saved # Between 1999 and 2009



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union





X1, X0, Z1, Z0 all come directly from dataprep object.


**************** 
 searching for synthetic control unit  
 

**************** 
**************** 
**************** 

MSPE (LOSS V): 2.660544 

solution.v:
 0.2752884 8.00985e-05 0.0006707994 0.4687482 0.2411453 0.008954685 0.005112477 

solution.w:
 1.59427e-05 1.04959e-05 1.19579e-05 0.1367322 1.12166e-05 2.60626e-05 3.83051e-05 0.0001724405 4.13242e-05 4.2473e-05 2.87778e-05 2.45509e-05 0.0006475294 5.92964e-05 8.7124e-06 0.208757 0.1690749 2.58467e-05 0.2739272 1.98343e-05 6.8578e-06 1.06507e-05 9.9014e-06 0.2102964 

$tab.pred
                                             Treated Synthetic Sample Mean
state.gdp.capita                              23.285    23.079      11.830
state.gdp.growth.percent                       1.330     2.585       3.528
population.projection.ln                      17.335    14.838      14.867
years.schooling.imp                            6.089     6.110       4.963
special.homicide.rates.1990.199

[1] 20331.17

In [1]:
library(bsts)

Warning message:
"package 'bsts' was built under R version 4.2.3"
Loading required package: BoomSpikeSlab

Warning message:
"package 'BoomSpikeSlab' was built under R version 4.2.3"
Loading required package: Boom

Warning message:
"package 'Boom' was built under R version 4.2.3"

Attaching package: 'Boom'


The following object is masked from 'package:stats':

    rWishart



Attaching package: 'BoomSpikeSlab'


The following object is masked from 'package:stats':

    knots


Loading required package: zoo

Warning message:
"package 'zoo' was built under R version 4.2.3"

Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: xts

Warning message:
"package 'xts' was built under R version 4.2.3"

################################### WARNING ###################################
# We noticed you have dplyr installed. The dplyr lag() function breaks how    #
# base R's lag() function is supposed to work, whic

In [2]:
# Read in the data
df <- read.csv("df.csv", header = TRUE)

# Specify the predictors for the model
predictors <- c("state.gdp.capita",
                "state.gdp.growth.percent",
                "population.projection.ln",
                "years.schooling.imp",
                "proportion.extreme.poverty",
                "gini.imp")


In [6]:
# Load necessary packages
library(bsts) # time series modeling

# Load data
df <- read.csv("df.csv", header = TRUE)

# Set up data for modeling
df <- df %>%
  filter(!is.na(homicide.rates)) %>% # remove missing values
  mutate(date = as.Date(paste0("01-01-", year))) %>% # create date column
  select(date, homicide.rates) # select relevant columns


In [7]:
# Set prior and control variables
prior.vars <- c("state.gdp.capita",
                "state.gdp.growth.percent",
                "population.projection.ln",
                "years.schooling.imp",
                "proportion.extreme.poverty",
                "gini.imp")

control.vars <- c("abbreviation == 'AC'",
                  "abbreviation == 'AL'",
                  "abbreviation == 'AM'",
                  "abbreviation == 'AP'",
                  "abbreviation == 'BA'",
                  "abbreviation == 'CE'",
                  "abbreviation == 'DF'",
                  "abbreviation == 'ES'",
                  "abbreviation == 'GO'",
                  "abbreviation == 'MA'",
                  "abbreviation == 'MG'",
                  "abbreviation == 'MS'",
                  "abbreviation == 'MT'",
                  "abbreviation == 'PA'",
                  "abbreviation == 'PB'",
                  "abbreviation == 'PE'",
                  "abbreviation == 'PI'",
                  "abbreviation == 'PR'",
                  "abbreviation == 'RJ'",
                  "abbreviation == 'RN'",
                  "abbreviation == 'RO'",
                  "abbreviation == 'RR'",
                  "abbreviation == 'RS'",
                  "abbreviation == 'SC'",
                  "abbreviation == 'SE'",
                  "abbreviation == 'SP'",
                  "abbreviation == 'TO'")

treat.var <- "abbreviation == 'SP'"


In [10]:
# Create input data for bsts model
bsts_data <- list()
bsts_data$y <- df$homicide.rates
bsts_data$xreg <- df %>% select(all_of(prior.vars))
bsts_data$state <- as.numeric(df %>% filter(eval(parse(text = treat.var))) %>% select(homicide.rates))
bsts_data$state.name <- as.character(df %>% filter(eval(parse(text = treat.var))) %>% select(abbreviation))


ERROR: [1m[33mError[39m in `select()`:[22m
[33m![39m Problem while evaluating `all_of(prior.vars)`.
[1mCaused by error in `all_of()`:[22m
[33m![39m Can't subset elements that don't exist.
[31m✖[39m Elements `state.gdp.capita`, `state.gdp.growth.percent`, `population.projection.ln`, `years.schooling.imp`, `proportion.extreme.poverty`, etc. don't exist.


In [14]:
names(df)

[1] "date"           "homicide.rates"